In [1]:
import torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.autograd import Variable
import torchvision
from PIL import Image

In [2]:
class chestxray():
    def __init__(self, csv_file, transform=None):
        self.transform = transform
        self.frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        img_name= self.frame.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        label = self.frame.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
from torchvision import transforms
batch_size = 16

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 128

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 50000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

class_num = 3

device = torch.device("cuda:2" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
# Decide which device we want to run on
from torchvision import transforms
import pandas as pd
normalize = transforms.Normalize(mean=[0.456, 0.456, 0.456], std=[0.225, 0.225, 0.225])
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256), transforms.ToTensor(), normalize])
dataset = chestxray('../training_gans.csv',transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [5]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = 1
        self.label_emb = nn.Embedding(3,3)
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz + 3, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16 
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d(    ngf,      int(ngf/2) , 4, 2, 1, bias=False),
            nn.BatchNorm2d(int(ngf/2)),
            nn.ReLU(True),
            # state size. (ngf/2) x 128 x 128
            nn.ConvTranspose2d(    int(ngf/2) ,   nc , 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 256 x 256
        )

    def forward(self, input, labels):
        z = input.view(input.size(0), nz, 1, 1)
        c = self.label_emb(labels).view(input.size(0), 3, 1, 1)
        x = torch.cat([z, c], 1)
        return self.main(x)

In [6]:
class Discriminator(nn.Module):
    def __init__(self, original_model):
        super(Discriminator, self).__init__()
        self.dnet= original_model
        self.fc = nn.Linear(1000, 100)
        self.dc = nn.Sequential(
            nn.Linear(100, 1),
            nn.Sigmoid(),
        )
        self.cl = nn.Sequential(
            nn.Linear(100, 3),
        )
        
    def forward(self, inp):
        x = self.dnet(inp)
        x = x.view(x.shape[0], 1, x.shape[1])
        x = self.fc(x)
        d = self.dc(x)
        c = self.cl(x)

        return d, c

In [7]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (label_emb): Embedding(3, 3)
  (main): Sequential(
    (0): ConvTranspose2d(131, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (

In [8]:
# Create the Discriminator
cnn = torchvision.models.densenet121(pretrained=True)
netD = Discriminator(cnn).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

print(netD)

Discriminator(
  (dnet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
netG = Generator(ngpu)
netG.load_state_dict(torch.load("generator_d_model",map_location="cuda:2"))
netG.to(device)
#netD = torch.load("discriminator_d")
#netD.to(device)

Generator(
  (label_emb): Embedding(3, 3)
  (main): Sequential(
    (0): ConvTranspose2d(131, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (

In [ ]:
import torch.optim as optim
import random
# Initialize BCELoss function
BCE_loss = nn.BCELoss().to(device)
CE_loss= nn.CrossEntropyLoss().to(device)

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = random.uniform(0, 0.2)
fake_label = random.uniform(0.8, 1)


# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

for param in netD.parameters():
     param.requires_grad = True

In [ ]:
# Training Loop

# Lists to keep track of progress
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, (data, labels) in enumerate(data_loader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output, pred = netD(real_cpu)
        #labels= torch.from_numpy(np.squeeze(np.eye(3)[labels.reshape(-1)]) ).to(device)
        labels= labels.long().to(device)
        # Calculate loss on all-real batch
        errD_real = BCE_loss(output, label)
        errC_real= CE_loss(pred.squeeze(), labels)
        # Calculate gradients for D in backward pass
        (errD_real+errC_real).backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = Variable(torch.randn(b_size, nz, 1, 1)).to(device)
        fake_labels = Variable(torch.LongTensor(np.random.randint(0, 3, b_size))).to(device)
        # Generate fake image batch with G
        fake = netG(noise, fake_labels)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output, pred = netD(fake.detach())
        # Calculate D's loss on the all-fake batch
        errD_fake = BCE_loss(output, label)
        errC_fake= CE_loss(pred.squeeze(), fake_labels)
        # Calculate the gradients for this batch
        (errD_fake+errC_fake).backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake +errC_real +errC_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output, pred = netD(fake)
        # Calculate G's loss based on this output
        errG = BCE_loss(output, label)
        errG_C= CE_loss(pred.squeeze(), fake_labels)
        errG+=errG_C
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

      #  while(errG - errD > 7):
       #     print("in the loop")
        #    netG.zero_grad()
         #   label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
        #    output = netD(fake, fake_labels).view(-1)
            # Calculate G's loss based on this output
        #    errG = criterion(output, label)
            # Calculate gradients for G
        #    errG.backward(retain_graph=True)
        #    D_G_z2 = output.mean().item()
            # Update G
        #    optimizerG.step()
            # Output training stats

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(data_loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
     #   if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
     #       with torch.no_grad():
     #           fake = netG(fixed_noise).detach().cpu()
      #      img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1
    torch.save(netG.state_dict(), "generator_d_model")
    #torch.save(netD.state_dict(), "discriminator_d_model")
    torch.save(netD,"discriminator_d")

Starting Training Loop...


/eerun/Python-3.7.2/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[0/50000][0/2231]	Loss_D: 3.5556	Loss_G: 1.2471	D(x): 0.4594	D(G(z)): 0.4596 / 0.6110
[0/50000][50/2231]	Loss_D: 1.8744	Loss_G: 2.2666	D(x): 0.1727	D(G(z)): 0.8532 / 0.8269
[0/50000][100/2231]	Loss_D: 1.2286	Loss_G: 4.5108	D(x): 0.0294	D(G(z)): 0.9576 / 0.9767
[0/50000][150/2231]	Loss_D: 1.2841	Loss_G: 5.5495	D(x): 0.0198	D(G(z)): 0.9098 / 0.9864
[0/50000][200/2231]	Loss_D: 1.1628	Loss_G: 5.4216	D(x): 0.0159	D(G(z)): 0.9169 / 0.9750
[0/50000][250/2231]	Loss_D: 1.4382	Loss_G: 4.2369	D(x): 0.0306	D(G(z)): 0.9488 / 0.9845
[0/50000][300/2231]	Loss_D: 1.6775	Loss_G: 4.1777	D(x): 0.0132	D(G(z)): 0.9509 / 0.9771
[0/50000][350/2231]	Loss_D: 1.2345	Loss_G: 3.1709	D(x): 0.0134	D(G(z)): 0.8513 / 0.9490
[0/50000][400/2231]	Loss_D: 1.3611	Loss_G: 4.0259	D(x): 0.0169	D(G(z)): 0.9747 / 0.9814
[0/50000][450/2231]	Loss_D: 1.5394	Loss_G: 3.7819	D(x): 0.0255	D(G(z)): 0.9587 / 0.9711
[0/50000][500/2231]	Loss_D: 1.1796	Loss_G: 4.1129	D(x): 0.0185	D(G(z)): 0.9813 / 0.9807
[0/50000][550/2231]	Loss_D: 1.3792	

/eerun/Python-3.7.2/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/eerun/Python-3.7.2/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[1/50000][0/2231]	Loss_D: 1.1943	Loss_G: 3.9652	D(x): 0.0144	D(G(z)): 0.9671 / 0.9809
[1/50000][50/2231]	Loss_D: 1.4121	Loss_G: 3.8170	D(x): 0.0090	D(G(z)): 0.9768 / 0.9782
[1/50000][100/2231]	Loss_D: 1.1746	Loss_G: 4.0227	D(x): 0.0130	D(G(z)): 0.9832 / 0.9826
[1/50000][150/2231]	Loss_D: 1.1071	Loss_G: 3.8177	D(x): 0.0097	D(G(z)): 0.9809 / 0.9785
[1/50000][200/2231]	Loss_D: 1.3351	Loss_G: 4.2301	D(x): 0.0085	D(G(z)): 0.9772 / 0.9836
[1/50000][250/2231]	Loss_D: 1.2035	Loss_G: 5.1622	D(x): 0.0076	D(G(z)): 0.9809 / 0.9591
[1/50000][300/2231]	Loss_D: 1.3137	Loss_G: 3.5239	D(x): 0.0878	D(G(z)): 0.9897 / 0.9647
[1/50000][350/2231]	Loss_D: 1.2533	Loss_G: 3.5971	D(x): 0.0112	D(G(z)): 0.9833 / 0.9733
[1/50000][400/2231]	Loss_D: 1.2124	Loss_G: 4.3507	D(x): 0.0039	D(G(z)): 0.9759 / 0.9864
[1/50000][450/2231]	Loss_D: 1.1940	Loss_G: 3.9722	D(x): 0.0158	D(G(z)): 0.9774 / 0.9815
[1/50000][500/2231]	Loss_D: 1.0509	Loss_G: 4.4398	D(x): 0.0109	D(G(z)): 0.9888 / 0.9881
[1/50000][550/2231]	Loss_D: 1.0063	

[3/50000][150/2231]	Loss_D: 1.1377	Loss_G: 3.8428	D(x): 0.0076	D(G(z)): 0.9775 / 0.9790
[3/50000][200/2231]	Loss_D: 1.0114	Loss_G: 3.9806	D(x): 0.0115	D(G(z)): 0.9832 / 0.9819
[3/50000][250/2231]	Loss_D: 1.3167	Loss_G: 3.9666	D(x): 0.0075	D(G(z)): 0.9764 / 0.9813
[3/50000][300/2231]	Loss_D: 1.1488	Loss_G: 3.8758	D(x): 0.0078	D(G(z)): 0.9802 / 0.9799
[3/50000][350/2231]	Loss_D: 1.0846	Loss_G: 3.8268	D(x): 0.0087	D(G(z)): 0.9749 / 0.9789
[3/50000][400/2231]	Loss_D: 0.9838	Loss_G: 3.8359	D(x): 0.0097	D(G(z)): 0.9763 / 0.9790
[3/50000][450/2231]	Loss_D: 1.0629	Loss_G: 3.8695	D(x): 0.0097	D(G(z)): 0.9714 / 0.9798
[3/50000][500/2231]	Loss_D: 1.0183	Loss_G: 3.8693	D(x): 0.0073	D(G(z)): 0.9767 / 0.9797
[3/50000][550/2231]	Loss_D: 1.0857	Loss_G: 3.8212	D(x): 0.0083	D(G(z)): 0.9774 / 0.9787
[3/50000][600/2231]	Loss_D: 1.0396	Loss_G: 3.8357	D(x): 0.0110	D(G(z)): 0.9793 / 0.9789
[3/50000][650/2231]	Loss_D: 1.2375	Loss_G: 3.7983	D(x): 0.0084	D(G(z)): 0.9765 / 0.9783
